In [1]:
import json
import logging
import requests
import pandas as pd
from pprint import pprint
from bs4 import BeautifulSoup

In [2]:
LOG_FILENAME = "my.log"
JSON_FILENAME = "my.json"
LOG_FORMAT = "%(asctime)s - %(levelname)s - %(message)s"
with open(LOG_FILENAME, 'w') as f:
    f.writelines([LOG_FORMAT, '\n'])
logging.basicConfig(filename=LOG_FILENAME, level=logging.DEBUG, format=LOG_FORMAT)

logging.debug("This is a debug log.")
logging.info("This is a info log.")
logging.warning("This is a warning log.")
logging.error("This is a error log.")
logging.critical("This is a critical log.")

In [3]:
response = requests.get("https://ncov.dxy.cn/ncovh5/view/pneumonia")
HTML = response.text.encode(response.encoding).decode()
soup = BeautifulSoup(HTML)
soup.prettify

<bound method Tag.prettify of <!DOCTYPE html>
<html lang="zh-cn" xmlns:layout="http://www.ultraq.net.nz/web/thymeleaf/layout"><head>
<link href="//assets.dxycdn.com/gitrepo/ncov-mobile/dist/umi.bundle.css?t=1580786525309" rel="stylesheet"/>
<meta charset="utf-8"/>
<meta content="width=device-width,initial-scale=1,user-scalable=0,viewport-fit=cover" name="viewport"/>
<meta content="#000000" name="theme-color"/>
<title>全国新型肺炎疫情实时动态 - 丁香园·丁香医生</title>
<script>
        window.routerBase = "/ncovh5/view";
    </script>
<script charset="utf-8" src="//assets.dxycdn.com/gitrepo/ncov-mobile/dist/vendors~p__Pneumonia~p__Pneumonia__recommend-list~p__Pneumonia__rumor-list~p__Pneumonia__timeline.async.5ccb549d.js"></script><script charset="utf-8" src="//assets.dxycdn.com/gitrepo/ncov-mobile/dist/vendors~p__Pneumonia~p__Pneumonia__rumor-list.async.9184546f.js"></script><link href="//assets.dxycdn.com/gitrepo/ncov-mobile/dist/vendors~p__Pneumonia.async.9f1a94c3.css" rel="stylesheet" type="text/css"/>

In [4]:
text = soup.contents[1].text
def parse_text(text):
    subtext = text[text.find("window.getAreaStat")::]
    subtext = subtext[subtext.find("[{")::]
    # print(subtext)
    num = 0
    chars = []
    for c in subtext:
        chars.append(c)
        if c == '[':
            num += 1
        if c == ']':
            num -= 1
        if num == 0:
            break
    return ''.join(chars)
counting_json = json.loads(parse_text(text))
print(counting_json)

[{'provinceName': '湖北省', 'provinceShortName': '湖北', 'confirmedCount': 13522, 'suspectedCount': 0, 'curedCount': 398, 'deadCount': 414, 'comment': '待明确地区，治愈 96', 'locationId': 420000, 'cities': [{'cityName': '武汉', 'confirmedCount': 6384, 'suspectedCount': 0, 'curedCount': 307, 'deadCount': 313, 'locationId': 420100}, {'cityName': '黄冈', 'confirmedCount': 1422, 'suspectedCount': 0, 'curedCount': 41, 'deadCount': 19, 'locationId': 421100}, {'cityName': '孝感', 'confirmedCount': 1120, 'suspectedCount': 0, 'curedCount': 4, 'deadCount': 17, 'locationId': 420900}, {'cityName': '随州', 'confirmedCount': 641, 'suspectedCount': 0, 'curedCount': 3, 'deadCount': 6, 'locationId': 421300}, {'cityName': '襄阳', 'confirmedCount': 632, 'suspectedCount': 0, 'curedCount': 1, 'deadCount': 1, 'locationId': 420600}, {'cityName': '荆州', 'confirmedCount': 613, 'suspectedCount': 0, 'curedCount': 6, 'deadCount': 7, 'locationId': 421000}, {'cityName': '宜昌', 'confirmedCount': 452, 'suspectedCount': 0, 'curedCount': 2, 'd

In [5]:
country_df = pd.read_json(json.dumps(counting_json))
country_df.to_json(open(JSON_FILENAME, 'w'))
country_df

,provinceName,provinceShortName,confirmedCount,suspectedCount,curedCount,deadCount,comment,locationId,cities
0,湖北省,湖北,13522,0,398,414,待明确地区，治愈 96,420000,"[{'cityName': '武汉', 'confirmedCount': 6384, 's..."
1,浙江省,浙江,829,0,48,0,,330000,"[{'cityName': '温州', 'confirmedCount': 340, 'su..."
2,广东省,广东,797,0,24,0,,440000,"[{'cityName': '深圳', 'confirmedCount': 269, 'su..."
3,河南省,河南,675,2,20,2,待明确地区：治愈 6；参考卫健委统计口径，部分县区与地级市合并,410000,"[{'cityName': '信阳', 'confirmedCount': 112, 'su..."
4,湖南省,湖南,593,0,26,0,,430000,"[{'cityName': '长沙', 'confirmedCount': 148, 'su..."
5,安徽省,安徽,480,0,14,0,,340000,"[{'cityName': '合肥', 'confirmedCount': 81, 'sus..."
6,江西省,江西,476,0,19,0,,360000,"[{'cityName': '南昌', 'confirmedCount': 121, 'su..."
7,重庆市,重庆,337,0,9,2,,500000,"[{'cityName': '万州区', 'confirmedCount': 56, 'su..."
8,江苏省,江苏,308,0,10,0,,320000,"[{'cityName': '苏州', 'confirmedCount': 52, 'sus..."
9,四川省,四川,282,0,14,1,待明确地区：治愈 1,510000,"[{'cityName': '成都', 'confirmedCount': 87, 'sus..."


In [6]:
provinces_df = dict()
for idx in country_df.index:
    se = country_df.iloc[idx]
    if se['cities']:
        provinces_df[se['provinceName']] = pd.read_json(json.dumps(se['cities']))
pprint(provinces_df)

{'上海市':    cityName  confirmedCount  suspectedCount  curedCount  deadCount  locationId
0    外地来沪人员              79               0           6          1          -1
1      浦东新区              40               0           0          0      310115
2       徐汇区              11               0           0          0      310104
3       宝山区              11               0           0          0      310113
4       闵行区              10               0           0          0      310112
5       静安区               9               0           0          0      310106
6       长宁区               8               0           0          0      310105
7       松江区               7               0           0          0      310117
8       普陀区               6               0           0          0      310107
9       虹口区               5               0           0          0      310109
10      黄浦区               5               0           0          0      310101
11      杨浦区               5               0 

In [7]:
with open(LOG_FILENAME, 'r') as f:
    for e in f.readlines():
        print(e, end='')

%(asctime)s - %(levelname)s - %(message)s
2020-02-04 12:29:15,018 - DEBUG - This is a debug log.
2020-02-04 12:29:15,019 - INFO - This is a info log.
2020-02-04 12:29:15,020 - WARNING - This is a warning log.
2020-02-04 12:29:15,022 - ERROR - This is a error log.
2020-02-04 12:29:15,023 - CRITICAL - This is a critical log.
2020-02-04 12:29:15,071 - DEBUG - Starting new HTTPS connection (1): ncov.dxy.cn:443
2020-02-04 12:29:15,197 - DEBUG - https://ncov.dxy.cn:443 "GET /ncovh5/view/pneumonia HTTP/1.1" 200 None
